In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
import os, glob, shutil
from DFTStructureGenerator import logfile_process, FormatConverter

In [4]:
new_num_df = pd.read_excel('New_number_BINOL.xlsx')
translation_dict = {old: new for new, old in zip(new_num_df['New_number'], new_num_df['Old_number'])}
old_space_df = pd.read_csv(r'Data\Iteration_2\Full_Space_ActiveLearning.csv')
old_space_binol = np.unique(old_space_df['Binol'].to_numpy())
old_space_ligand = np.unique(old_space_df['Ligand'].to_numpy())
len(old_space_ligand), len(old_space_binol)

(178, 44)

In [6]:
assert np.all(np.isin(old_space_binol, new_num_df['Old_number']))
assert np.all(np.isin(old_space_ligand, new_num_df['Old_number']))


In [8]:
data_csv = pd.read_csv("Data_clear_with_sites.csv")
data_csv['Index'] = data_csv['Index'].map(translation_dict)
data_csv.drop(columns=['Type_Index'], inplace=True)
data_csv = data_csv.dropna().reset_index(drop=True)
data_csv['Index'] = data_csv['Index'].astype(int)
data_csv.sort_values(by=['Index'], inplace=True)
data_csv.to_csv('Data_clear_with_sites.csv', index=False)

In [13]:
csv = 'BINOL_result_Fig4_L1009.csv'
df = pd.read_csv(csv)
df['Binol'] = df['Binol'].map(translation_dict)
df['Ligand'] = df['Ligand'].map(translation_dict)
df.to_csv(csv, index=False)

PermissionError: [Errno 13] Permission denied: 'BINOL_result_Fig4_L1009.csv'

In [17]:
for row_id, row in new_num_df.iterrows():
    new, old = row['New_number'], row["Old_number"]
    for subdir in ['GS_OPT', 'Mols', "newGS", 'newmols']:
        if not os.path.isdir(f'Data/DFT/{subdir}'):
            os.mkdir(f'Data/DFT/{subdir}')
        files = glob.glob(f"ignored/backup/{subdir}/{old:05}*")
        for each in files:
            new_file = os.path.join(f'Data/DFT/{subdir}', os.path.basename(each).replace(f'{old:05}', f'{new:05}'))
            shutil.copy(each, new_file)

In [ ]:
for row_id, row in new_num_df.iterrows():
    new, old = row['New'], row["Old"]
    paths = glob.glob(f"ignored/backup/Mol_xtb/*/{old:05}*")
    for each in paths[:0]:
        new_path = os.path.join(f'Data/DFT/Mol_xtb/charge_0_0', os.path.basename(each).replace(f'{old:05}', f'{new:05}'))
        # shutil.copytree(each, new_path)

In [18]:
import pickle
with open(r"Data/all_fp_map.pkl", 'rb')as f:
    qm_dict, area_dict = pickle.load(f)

In [20]:
new_qm_dict = {row['New_number']: qm_dict[row['Old_number']] for _, row in new_num_df.iterrows()}
new_area_dict = {row['New_number']: area_dict[row['Old_number']] for _, row in new_num_df.iterrows()}

KeyError: 0

In [ ]:
# with open(r"Data/all_fp_map.pkl", 'wb')as f:
#     pickle.dump([new_qm_dict, new_area_dict], f)

In [47]:
for csv in glob.glob("Data/Iteration_2/Result/*.csv"):
    df = pd.read_csv(csv)
    # print(len(df))
    for name in ['Binol_smiles', 'Ligand_smiles', 'Binol_conf_id', 'Ligand_conf_id', 'Binol_Sites', 'Ligand_Sites', 'Binol_G', 'Ligand_G', 'Unnamed: 4']:
        if name in df.columns:
            df = df.drop(columns = name)
    df["Binol"] = df["Binol"].map(translation_dict)
    df["Ligand"] = df["Ligand"].map(translation_dict)
    # df = df.dropna().reset_index(drop=True)
    df['Binol'] = df['Binol'].astype(int)
    df['Ligand'] = df['Ligand'].astype(int)

    df.to_csv(csv.replace('Result', "Result_"), index=False)

In [43]:
for id_ in range(1,16):
    csv = f"Data/Iteration_2/Result/BINOL_result_{id_:04}.csv"
    new_csv = f"Data/Iteration_2/Result/BINOL_result_{id_ - 1:04}.csv"
    shutil.move(csv, new_csv)

In [48]:
for id_ in range(15):
    csv = f"Data/Iteration_2/Result/BINOL_result_{id_:04}.csv"
    newcsv = f"Data/Iteration_2/Iteration_Data/iter_{id_:05}.csv"
    df = pd.read_csv(csv)
    df.drop(columns=['R', 'S'], inplace=True, errors='ignore')
    df.to_csv(newcsv, index=False)

In [49]:
AL_data = pd.read_csv("Data/Iteration_2/Full_Space_ActiveLearning.csv")
AL_id = [f'{row.Binol:05}_{row.Ligand:05}' for _, row in AL_data.iterrows()]


In [50]:
for id_ in range(15):
    csv = f"Data/Iteration_2/Iteration_Data/iter_{id_:05}.csv"
    df = pd.read_csv(csv)
    temp_id = [f'{int(row.Binol):05}_{int(row.Ligand):05}' for _, row in df.iterrows()]
    collect_id = [AL_id.index(t_id) for t_id in temp_id]
    print(len(collect_id))
    np.save(f"Data/Iteration_2/Iteration_Data/iter_{id_:05}.npy", np.array(collect_id))
    # break

50
50
50
50
50
50
50
50
50
50
50
50
50
50
50


# CuBox

In [244]:
methods = [
    '#p b3lyp nosymm def2tzvpp em=gd3bj scrf=(smd,solvent=dichloromethane)',
    '#p M06 nosymm def2tzvpp scrf=(smd,solvent=dichloromethane)', 
    '#p m062x nosymm def2tzvpp scrf=(smd,solvent=dichloromethane)', 
    '#p wb97xd nosymm def2tzvpp scrf=(smd,solvent=dichloromethane)', 
    '#p pbe1pbe nosymm def2tzvpp em=gd3bj scrf=(smd,solvent=dichloromethane)'
]
names = ['b3lyp', 'M06', 'm062x', 'wb97xd', 'pbe0']
target_dir = 'ignored/DFT/b3def2svp'
for x in ['R', "S"]:
    log = logfile_process.Logfile(os.path.join(target_dir, f'{x}.log'))
    for name, method in zip(names, methods):
        FormatConverter.block_to_gjf(log.symbol_list, log.running_positions[-1], os.path.join(target_dir, f'{x}_{name}.gjf'), method=method, charge=log.charge, multiplicity=log.multiplicity)
    

Can't find title
Can't find title


In [245]:
all_opt_dirs = glob.glob("ignored/DFT/*")
result_df = pd.DataFrame(columns=['OPT_method', 'SPE_method', 'R_G', 'S_G', 'deltaG'])
for each_dir in all_opt_dirs:
    if not os.path.isdir(each_dir):
        continue
    R_log_file = os.path.join(each_dir, "R.log")
    S_log_file = os.path.join(each_dir, "S.log")
    if not (os.path.isfile(R_log_file) and os.path.isfile(S_log_file)):
        continue
    R_log = logfile_process.Logfile(R_log_file)
    S_log = logfile_process.Logfile(S_log_file)
    if not R_log.normal_end or not S_log.normal_end:
        continue
    R_G_cor = R_log.all_engs[-1]
    S_G_cor = S_log.all_engs[-1]

    result_df = result_df._append({
        "OPT_method": each_dir.split("\\")[-1],
        "SPE_method": "\\",
        "R_G": R_G_cor + R_log.all_engs[0],
        "S_G": S_G_cor + S_log.all_engs[0],
        "deltaG": (R_G_cor + R_log.all_engs[0] - S_G_cor - S_log.all_engs[0]) * 627.5
    }, ignore_index=True)

    R_spe_logs = glob.glob(os.path.join(each_dir, "R_*.log"))
    for R_spe_log in R_spe_logs:
        spe_name = os.path.basename(R_spe_log).split("_")[1].split(".")[0]
        S_spe_log = R_spe_log.replace("R_", "S_")
        if not os.path.isfile(S_spe_log):
            continue
        R_spe_log = logfile_process.Logfile(R_spe_log)
        S_spe_log = logfile_process.Logfile(S_spe_log)
        if not R_spe_log.normal_end or not S_spe_log.normal_end:
            continue
        R_spe_eng = R_spe_log.all_engs[-1]
        S_spe_eng = S_spe_log.all_engs[-1]
        R_G = R_spe_eng + R_G_cor
        S_G = S_spe_eng + S_G_cor
        deltaG = R_G - S_G
        result_df = result_df._append({
            "OPT_method": each_dir.split("\\")[-1],
            "SPE_method": spe_name,
            "R_G": R_G,
            "S_G": S_G,
            "deltaG": deltaG * 627.5
            
        }, ignore_index=True)

        

Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
ignored/DFT\b3def2svp\R_M062x.log didn't run successful
ignored/DFT\b3def2svp\R_M062x.log 应该是没跑完
ignored/DFT\b3def2svp\R_M062x.log, can't find any engs
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
ignored/DFT\b3def2svp(solvent)\R_m062x.log didn't run successful
Can't find title
Can't find title
Can't find title
Can't find title
Can't find title
ignored/DFT\m062x_def2svp(solvent)\S.log didn't run successful
Can't find title


In [246]:
result_df

,OPT_method,SPE_method,R_G,S_G,deltaG
0,1_4kcal,\,-3707.420586,-3707.422692,1.321327
1,1_4kcal,sp,-3709.967456,-3709.969649,1.375894
2,2_0kcal,\,-3707.419553,-3707.422692,1.969597
3,2_0kcal,m06l,-3709.966430,-3709.969649,2.019904
4,b3def2svp,\,-3707.949200,-3707.960094,6.836067
5,b3def2svp,b3lyp,-3710.571368,-3710.577880,4.086399
6,b3def2svp,M06,-3708.755541,-3708.760456,3.083949
7,b3def2svp,pbe0,-3707.773672,-3707.780540,4.309701
8,b3def2svp,wb97xd,-3709.661171,-3709.667288,3.838850
9,b3def2svp(solvent),\,-3708.007841,-3708.017694,6.182808


In [247]:
new_num_df = pd.read_excel('New_number.xlsx')
translation_dict = {old: new for new, old in zip(new_num_df['New'], new_num_df['Old'])}
new_2_num_df = pd.read_excel('New_number_BINOL.xlsx')
new_2_translation_dict = {translation_dict[old]: new for new, old in zip(new_2_num_df['New_number'], new_2_num_df['Old_number'])}

KeyError: 140

In [249]:
translation_dict.keys()

dict_keys([1001, 1012, 1015, 1153, 1184, 1038, 1171, 1051, 1069, 1077, 1173, 1063, 1156, 1203, 1158, 1161, 1166, 1167, 1192, 1194, 1092, 1003, 1037, 1182, 1017, 1040, 1195, 1028, 1029, 1183, 1066, 1190, 1041, 1075, 1068, 1064, 1070, 1080, 1165, 1085, 1094, 1095, 1096, 1098, 1050, 1036, 1033, 1016, 1151, 1039, 1022, 1025, 1024, 1059, 1065, 1188, 1044, 1177, 1205, 1005, 1048, 1162, 1097, 1083, 1078, 1168, 1009, 1174, 1031, 1154, 1019, 1010, 1176, 1197, 1200, 1034, 1155, 1175, 1071, 1187, 1204, 1202, 1046, 1052, 1191, 1193, 1169, 1170, 1159, 1004, 1178, 1014, 1152, 1179, 1196, 1030, 1199, 1172, 1062, 1072, 1076, 1073, 1157, 1067, 1054, 1045, 1084, 1086, 1164, 1079, 1100, 1006, 1011, 1032, 1020, 1027, 1026, 1023, 1180, 1181, 1189, 1074, 1049, 1099, 1163, 1081, 1082, 1185, 1201, 1198, 1035, 1186, 1061, 1056, 1057, 1091, 1093, 1087, 1058, 1101, 1102, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1113, 1114, 1117, 1118, 1122, 1125, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1136, 1137,

In [250]:
translation_dict.keys()

dict_keys([1001, 1012, 1015, 1153, 1184, 1038, 1171, 1051, 1069, 1077, 1173, 1063, 1156, 1203, 1158, 1161, 1166, 1167, 1192, 1194, 1092, 1003, 1037, 1182, 1017, 1040, 1195, 1028, 1029, 1183, 1066, 1190, 1041, 1075, 1068, 1064, 1070, 1080, 1165, 1085, 1094, 1095, 1096, 1098, 1050, 1036, 1033, 1016, 1151, 1039, 1022, 1025, 1024, 1059, 1065, 1188, 1044, 1177, 1205, 1005, 1048, 1162, 1097, 1083, 1078, 1168, 1009, 1174, 1031, 1154, 1019, 1010, 1176, 1197, 1200, 1034, 1155, 1175, 1071, 1187, 1204, 1202, 1046, 1052, 1191, 1193, 1169, 1170, 1159, 1004, 1178, 1014, 1152, 1179, 1196, 1030, 1199, 1172, 1062, 1072, 1076, 1073, 1157, 1067, 1054, 1045, 1084, 1086, 1164, 1079, 1100, 1006, 1011, 1032, 1020, 1027, 1026, 1023, 1180, 1181, 1189, 1074, 1049, 1099, 1163, 1081, 1082, 1185, 1201, 1198, 1035, 1186, 1061, 1056, 1057, 1091, 1093, 1087, 1058, 1101, 1102, 1104, 1105, 1106, 1107, 1108, 1109, 1110, 1111, 1113, 1114, 1117, 1118, 1122, 1125, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1136, 1137,

In [3]:
# 把63的分子提取出来
target_files = glob.glob(f"Data/DFT/*/*{63:05}*")
new_dir = glob.glob("ignored/backup2/*")
for file_ in target_files:
    new_file = file_.replace("Data/DFT", "ignored/backup2").replace(f'{63:05}', f'{273:05}')
    if not os.path.isdir(os.path.dirname(new_file)):
        os.mkdir(os.path.dirname(new_file))
    shutil.copyfile(file_, new_file)

In [257]:
new_file

'BINOL_result_Fig4_L1009.csv'